In [1]:
import chess
from stockfish import Stockfish
import re
import requests
from ollama import chat
from ollama import ChatResponse
import json
import pandas as pd


In [2]:
stockfish = Stockfish()

In [3]:
# URL to scrape the text from.
url = "https://wtharvey.com/inst.html"

# Download the text from the website.
response = requests.get(url)
if response.status_code == 200:
    text = response.text
else:
    print(f"Failed to retrieve content. Status code: {response.status_code}")
    text = ""

# Regular expression pattern for matching FEN strings.
fen_pattern = re.compile(
    r'([prnbqkPRNBQK\d/]+\s+[wb]\s+(?:K?Q?k?q?|-)\s+(?:[a-h][36]|-)\s+\d+\s+\d+)'
)

# Find all FEN notations in the text.
fen_notations = fen_pattern.findall(text)

# Print the extracted FEN notations.
print("Extracted FEN Notations:")
# for fen in fen_notations:
#     print(fen)

print(fen_notations[:3])

Extracted FEN Notations:
['3Rr1k1/p5pn/1p3p2/7P/1PP1q3/2B5/PK1Q2r1/7R w - - 1 0', '4r1k1/p4p1p/1p1P2p1/1P5n/q4P2/8/2R2Q1P/4R2K b - - 0 1', '1k6/ppp4p/4P1p1/3p1n2/1P1P4/2P1P3/P2N1rqP/2KQ1R2 b - - 0 1']


In [4]:
fen_notations_ten = fen_notations[:10]

len(fen_notations_ten)

print(fen_notations_ten[0])

3Rr1k1/p5pn/1p3p2/7P/1PP1q3/2B5/PK1Q2r1/7R w - - 1 0


In [5]:
def valid_move_to_string(board):
    legal_moves = [
        board.san(move)
        for move in board.legal_moves
]
    
    moves = str(legal_moves[0])  # Start with the first move

    for move in legal_moves[1:]:  # Skip the first move and iterate over the rest
        moves = moves + " " + move
    return moves

In [6]:
def get_qwen_move(board:chess.Board, fen:str):
    valid_moves = valid_move_to_string(board)
    
    response: ChatResponse = chat(model='qwen:32b', messages=[
        {
            'role': 'user',
            'content': f"""You are playing chess and it is your turn.

    This is the current state of the game. Use this to work out where the pieces are on the board:

    FEN: {fen}

    The possible list of legal moves in SAN notation are: {valid_moves}. You have to choose a move from this list. Do not choose a move that is not in this list.
    
    Output the best move in SAN format to follow this position. Use the following single blob of JSON. Do not include any other information.

    {{
    "san": "The move in SAN format",
    "reason": "Why this is a good move"
    }}
    """
            }
        ])
    
    try:
        parsed_json = json.loads(response.message.content)
        qwen_move = parsed_json["san"]
    except Exception as e:
        print(f"Wrong format given by Qwen. \nQwen Output: {response.message.content}")

    

    return qwen_move

In [7]:
results = []

In [8]:
puzzle_id = 1

for fen in fen_notations_ten:
    for i in range (100):
        if not stockfish.is_fen_valid(fen): exit
        
        print(f"Processing puzzle_id: {puzzle_id} Iteration: {i}\nPosition: {fen}")

        board_stockfish = chess.Board(fen=fen)
        board_qwen = chess.Board(fen=fen)

        stockfish.set_fen_position(fen_position=fen)

        best_move = stockfish.get_best_move()

        board_stockfish.push(chess.Move.from_uci(best_move))
        # best_eval_info = stockfish.get_evaluation()
        best_eval = stockfish.get_evaluation()["value"]  # evaluation in centipawns (or mate score)
        print(f"Stockfish best move: {best_move} with evaluation: {best_eval}")




        qwen_move = get_qwen_move(board_qwen, fen)

        board_qwen.parse_san(qwen_move)

        # chess.Move.from_uci(board_qwen.parse_san(qwen_move))

        # print(f"QWEN MOVE: {qwen_move}\nQWEN MOVE TYPE: {type(qwen_move)}")

        # try:
        #     print(board_qwen.legal_moves)
        #     chess.Move.from_uci(board_qwen.parse_san(qwen_move))
        # except Exception as e:
        #     print(f"Invalid move geenrated by Qwen. The move is: {qwen_move}")
        #     print(f"Exception type: {type(e)}")
        #     continue

        stockfish.set_fen_position(fen_position=board_qwen.fen())
        provided_eval = stockfish.get_evaluation()["value"]

        score_difference = best_eval - provided_eval

        results.append({
            "Iteration": i,
            "Puzzle ID": puzzle_id,
            "Method": "Qwen 32B",
            "Best Move Score": best_eval,
            "Move made by Qwen:32B Score": provided_eval,
            "Score Difference": score_difference
        })
    
    puzzle_id += 1


Processing puzzle_id: 1 Iteration: 0
Position: 3Rr1k1/p5pn/1p3p2/7P/1PP1q3/2B5/PK1Q2r1/7R w - - 1 0
Stockfish best move: d8e8 with evaluation: 676
Processing puzzle_id: 1 Iteration: 1
Position: 3Rr1k1/p5pn/1p3p2/7P/1PP1q3/2B5/PK1Q2r1/7R w - - 1 0
Stockfish best move: d8e8 with evaluation: 676
Processing puzzle_id: 1 Iteration: 2
Position: 3Rr1k1/p5pn/1p3p2/7P/1PP1q3/2B5/PK1Q2r1/7R w - - 1 0
Stockfish best move: d8e8 with evaluation: 676
Processing puzzle_id: 1 Iteration: 3
Position: 3Rr1k1/p5pn/1p3p2/7P/1PP1q3/2B5/PK1Q2r1/7R w - - 1 0
Stockfish best move: d8e8 with evaluation: 676


KeyboardInterrupt: 

In [ ]:
df = pd.DataFrame(results)
df.to_csv("evaluation_results.csv", index=False)
print("Results saved to evaluation_results.csv")

In [ ]:
f"""You are playing chess and it is your turn.

    This is the current state of the game. Use this to work out where the pieces are on the board:

    FEN: {fen}

    The possible set of legal moves are: Bxe7 Bh6 Bf6 Bh4 Bf4 Be3 Bd2 Bc1 Nxf7 Nd7 Ng6 Nc6 Ng4 Nf3 Nxe7 Nc7+ Nf6+ Nb6 Nf4 Nb4 Ne3 Nc3 Ba6 Bb5+ Bb3 Rg1 Rf1 Kd2 Kf1 Kxd1 Rxd1 Rc1 Rb1 O-O h3 g3 f3 c3 b3 a3 h4 g4 f4 b4 a4. You have to choose one from this list. Do not choose a move that is not in this list.
    
    Output the best move in SAN format to follow this position. Use the following single blob of JSON. Do not include any other information.

    {{
    "san": "The move in SAN format",
    "reason": "Why this is a good move"
    }}
    """